In [14]:
import numpy as np 
import cv2
from PIL import Image

In [15]:
def get_limits(color):
    c = np.uint8([[color]])  # BGR values
    hsvC = cv2.cvtColor(c, cv2.COLOR_BGR2HSV)

    hue = hsvC[0][0][0]  # Get the hue value

    # Handle red hue wrap-around
    if hue >= 165:  # Upper limit for divided red hue
        lowerLimit = np.array([hue - 10, 100, 100], dtype=np.uint8)
        upperLimit = np.array([180, 255, 255], dtype=np.uint8)
    elif hue <= 15:  # Lower limit for divided red hue
        lowerLimit = np.array([0, 100, 100], dtype=np.uint8)
        upperLimit = np.array([hue + 10, 255, 255], dtype=np.uint8)
    else:
        lowerLimit = np.array([hue - 10, 100, 100], dtype=np.uint8)
        upperLimit = np.array([hue + 10, 255, 255], dtype=np.uint8)

    return lowerLimit, upperLimit

In [22]:
# Elegimos el color
yellow = [0, 255, 255]

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()

    # Cambiamos a hsv
    img_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # definimos los limites sueperiores y inferiores del color
    lowerLimit, upperLimit = get_limits(color=yellow)

    # creamos una mascara par detectar el color
    mask = cv2.inRange(img_hsv, lowerLimit, upperLimit)

    # transformarmos la informacion de la mascara a un array
    mask_ = Image.fromarray(mask)
    
    # obtenemos los box, la caja de la informacion del arrray
    bbox = mask_.getbbox()

    # pintamos la caja en cada imagen del video
    if bbox is not None:
        x1, y1, x2, y2 = bbox
        
        frame = cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 5)
    
    # giramos la camara para que se vea mejor
    flipped_frame = cv2.flip(frame, 2)

    cv2.imshow('frame', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()